# What Is Singleton?

> **A singleton is an identity which is embodied by exactly one coin** at any given time, and where it’s possible for coins to validate that they’re interacting with the current representative of that identity. It’s similar to, but simpler than, coloured coins. An NFT is barely more than a raw singleton. Singletons are also the foundation of rate limited wallets, distributed identities, pool protocol, and price oracles. -- [A Vision for DeFi in Chia](https://www.chia.net/2021/07/13/a-vision-for-defi-in-chia.en.html)

> **A singleton is a coin that is verifiably unique**. Similar to (but more powerful than) NFTs, a singleton cannot be copied, duplicated, or recreated if it is destroyed. It is a common outer puzzle for things like DIDs, NFTs, or the pooling protocol. -- [The Great Chia Glossary | Singleton](https://chialisp.com/docs/glossary)

> **A singleton is a special kind of puzzle that asserts uniqueness**, preventing other coins from being mistaken for it. It can change state over time, each time creating a new puzzle based on the previous. It wraps an `inner puzzle` which can have whatever behavior you desire. These traits combined allow for smart contracts and things like NFTs to be created. -- [What is a singleton?](https://developers.chia.net/t/what-is-a-singleton/87)

## Why Do We Need Singleton?
> Coins on the network are state, the puzzle is the data contained within them. You can spend them to change their state however you like, by creating new coins. This is what singletons excel at. -- [Can you store state on the network?](https://developers.chia.net/t/can-you-store-state-on-the-network/84)

In the account model (e.g, Ethereum), you have **a unique contract address** that you can access and update data. In the coin set model, however, coins are state, so we need to be able to extract state that we are interested from those coins somehow. To update those states, we will need to spend them and create new coins. 

### Coin Id & Puzzle Hash
Each coin has a unique [coin_id](../../basic/coins/notebook.ipynb) which derived from `parent_coin_id`, `puzzle_hash`, and its `amount` (in mojos). 

Once you spend the coin with the existing state, you could get a new unspent coin with the new state (different or same puzzle hash). However, this means that you have to keep track of a valid `coin_id` and/or `puzzle hash`. In addition, someone can just create new coins with the same puzzle hash out of nowhere which could be more confusing.

### A Singleton Puzzle & Launcher Id
It's clear that we need a way to model the idea of **unique id**, so the Chia Network comes up with [singleton puzzle](https://chialisp.com/docs/puzzles/singletons). **A singleton coin provides a unique id called launcher id that can be used as the permanent unique address.** The `launcher id` is basically the `coin_id` of the launcher coin that creates the first singleton coin. When a singleton coin is spent, at most one new singleton coin can be created. We can always track all of those singleton coins and find the unspent singleton coin by following the `launcher id`.


## Singleton Puzzle

The singleton puzzle is an outer puzzle that can wrap any inner puzzle. This ensures that all singleton coins follow the rules of singleton.

### Rules of Singleton
- A singleton must have an odd mojo value.
- Only one output (i.e., a singleton) coin can have an odd mojo value.
- All singletons should have the same format.

### Boot Strapping

To create **the first singleton coin (eve)**, we spend a standard coin and a [singleton launcher coin](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_launcher.clvm).

- The first singleton coin (eve) must be created from the coin called the Launcher by passing in our `singleton_puzzle_hash` (as a solution).
- `singleton_puzzle_hash` is a [singleton top layer puzzle](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.clvm) curried in `SINGLETON_STRUCT` and `INNER_PUZZLE`.
- `SINGLETON_STRUCT` is `(SINGLETON_MOD_HASH . (LAUNCHER_ID . SINGLETON_LAUNCHER_HASH))`.
- `SINGLETON_MOD_HASH` is a puzzle hash of the [singleton top layer puzzle](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.clvm).
- To make sure that the `singleton_puzzle_hash` is not tampered, we will utilize annoucement which contains the `singleton_puzzle_hash`.
- The starting coin (which can be a standard transaction coin) creates the launcher coin and assert the announcement from the same launcher coin that is spent at the same time. The launcher coin is an **ephemeral** coin.

<img src="bootstrapping_singleton.jpg" alt="Boot Strapping Singleton" width="600"/>


In [13]:
# https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.py
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

print(f'SINGLETON_MOD_HASH:\t\t{singleton_top_layer.SINGLETON_MOD_HASH}')
print(f'SINGLETON_LAUNCHER_HASH:\t{singleton_top_layer.SINGLETON_LAUNCHER_HASH}')


SINGLETON_MOD_HASH:		24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd
SINGLETON_LAUNCHER_HASH:	eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9


In [17]:
# starting coin
import json
def print_json(dict):
    print(json.dumps(dict, sort_keys=True, indent=4))

from cdv.test import Network, Wallet

network: Network = await Network.create()
await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

starting_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())
coin_records = await network.sim_client.get_coin_records_by_puzzle_hash(starting_puzzle.get_tree_hash())
starting_coin = next(cr.coin for cr in coin_records if cr.spent == False)
print_json(starting_coin.to_json_dict())


{
    "amount": 250000000000,
    "parent_coin_info": "0x27ae41e4649b934ca495991b7852b85500000000000000000000000000000001",
    "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
}


## Use Cases

### Plot NFT for Pooling
The most well-known singleton example is the [plot nft](https://github.com/Chia-Network/chia-blockchain/wiki/Pooling-FAQ#what-is-a-plot-nft), which keeps the pool that the farmer belongs to. Farmer can always switch the pool by spending the singleton plot nfg and provide the new pool information. 

> The original pool design was having pool keys in plot files, but that means if farmers want to switch pool, they'd have to re-plot!! [Pools in Chia](https://www.chia.net/2020/11/10/pools-in-chia.html)

### Others

Anything that needs to be unique on the blockchain can also be represented by a singleton.

- NFT (Non-Fungible Token) 
- DIDs (Decentralized Identifiers) : A parent singleton can be treated as DID.

## How it works?

To ensure that the coin is unique, when a singleton coin is spent, it creates another singleton coin with the same launcher id. This, however, doesn't prevent a singleton coin to create non-singleton coins.


